实现池化层的正向传播

In [21]:
import torch 
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size # 获取池化层核的大小（高、宽）
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)) # 初始化全零的输出，以便后面填充
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]): # 两层for循环，从左到右，从上到下的滑动
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max() # 最大池化
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean() # 平均池化
    return Y

验证二维最大池化层的输出

In [22]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
print(X.shape)
pool2d(X, (2, 2)) # 2*2的池化层：知道输入维度，卷积核（或者池化层大小），怎么计算输出层大小呢？答：输入维度-卷积核+1

torch.Size([3, 3])


tensor([[4., 5.],
        [7., 8.]])

验证平均池化层

In [23]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

填充和步幅

In [24]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

深度学习框架中的步幅与池化窗口的大小相同：意味着每次滑动窗口的时候，是不重叠的

In [25]:
pool2d = nn.MaxPool2d(3) # 这个3表示3*3的池化层，而且，不写步幅，则默认为也是3
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定

In [26]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2) # 3表示池化层大小3*3，
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

设定一个任意大小的矩形池化窗口，并分别设定填充和步幅的高度和宽度

In [27]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

池化层在每个输入通道上单独运算

In [28]:
X = torch.cat((X, X + 1), 1) # 两个通道
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [29]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2) # 每个通道是单独进行的
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])